In [ ]:
import warnings

import lightning.pytorch as pl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from lightning.pytorch.callbacks import RichProgressBar
from pytorch_forecasting import TimeSeriesDataSet
from pytorch_lightning.loggers import TensorBoardLogger
from sklearn.metrics import (
    explained_variance_score,
    mean_absolute_error,
    mean_squared_error,
    mean_squared_log_error,
    median_absolute_error,
    r2_score,
)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.svm import SVR
from torch.autograd import Variable
from torchinfo import summary
from torchmetrics import (
    ExplainedVariance,
    MeanAbsoluteError,
    MeanSquaredError,
    R2Score,
)

warnings.filterwarnings("ignore")

# config
%matplotlib inline
%config InlineBackend.figure_format='retina'

### 读取数据集


In [ ]:
nsdk_data = pd.read_csv("../data/A纳斯达克综合指数.csv", index_col=0, parse_dates=True)
dqs_data = pd.read_csv("../data/A道琼斯工业平均指数.csv", index_col=0, parse_dates=True)
bzpr_data = pd.read_csv("../data/A标准普尔500指数.csv", index_col=0, parse_dates=True)
sh_data = pd.read_csv("../data/A上证指数.csv", index_col=0, parse_dates=True)
sz_data = pd.read_csv("../data/A深证指数.csv", index_col=0, parse_dates=True)
cyb_data = pd.read_csv("../data/A创业板指数.csv", index_col=0, parse_dates=True)

In [ ]:
stock_market = [0, nsdk_data, dqs_data, bzpr_data, sh_data, sz_data, cyb_data]
stock_name = [
    "",
    "Nasdaq Composite Index",
    "DowJones Industrial Average Index",
    "S&P 500 Index",
    "Shanghai Composite Index",
    "Shenzhen Index",
    "ChiNext Index",
]

features_one = ["Open", "High", "Low", "Volume"]
features_two = [
    "open",
    "high",
    "low",
    "volume",
    "price_change",
    "p_change",
    "ma5",
    "ma10",
    "ma20",
    "v_ma5",
    "v_ma10",
    "v_ma20",
]

### DataModule


#### Univariate Time Series

In [ ]:
class UnivariateSeriesDataModule(pl.LightningDataModule):
    def __init__(
        self,
        data: pd.DataFrame,
        n_lags: int,
        horizon: int,
        test_size: float = 0.2,
        batch_size: int = 16,
        feature_name: str = "",
    ) -> None:
        super().__init__()
        self.data = data
        self.n_lags = n_lags
        self.horizon = horizon
        self.test_size = test_size
        self.batch_size = batch_size
        self.feature_name = feature_name
        self.training: TimeSeriesDataSet
        self.validation: TimeSeriesDataSet
        self.test: TimeSeriesDataSet
        self.predict: TimeSeriesDataSet
        self.target_scaler = MinMaxScaler()
        self.setup()

    def preprocess_data(self):
        self.data["target"] = self.data[self.feature_name]
        self.data["time_index"] = np.arange(len(self.data))
        self.data["group_id"] = 0

    def split_data(self):
        time_indices = self.data["time_index"].values
        train_indices, test_indices = train_test_split(
            time_indices, test_size=self.test_size, shuffle=False
        )
        train_indices, val_indices = train_test_split(
            train_indices, test_size=0.1, shuffle=False
        )
        return train_indices, val_indices, test_indices

    def scale_target(self, df, indices):
        scaled_values = self.target_scaler.transform(df.loc[indices, ["target"]])
        df.loc[indices, "target"] = scaled_values

    def setup(self, stage=None) -> None:
        self.preprocess_data()
        train_indices, val_indices, test_indices = self.split_data()

        train_df = self.data.loc[self.data["time_index"].isin(train_indices)]
        val_df = self.data.loc[self.data["time_index"].isin(val_indices)]
        test_df = self.data.loc[self.data["time_index"].isin(test_indices)]
        predict_df = pd.concat([train_df, val_df, test_df])

        self.target_scaler.fit(train_df[["target"]])
        self.scale_target(train_df, train_df.index)
        self.scale_target(val_df, val_df.index)
        self.scale_target(test_df, test_df.index)

        self.training = TimeSeriesDataSet(
            train_df,
            time_idx="time_index",
            target="target",
            group_ids=["group_id"],
            max_encoder_length=self.n_lags,
            max_prediction_length=self.horizon,
            time_varying_unknown_reals=[self.feature_name],
            scalers={name: MinMaxScaler() for name in self.feature_name},
        )
        self.validation = TimeSeriesDataSet.from_dataset(
            self.training, val_df, stop_randomization=True
        )
        self.test = TimeSeriesDataSet.from_dataset(
            self.training, test_df, stop_randomization=True
        )
        self.predict = TimeSeriesDataSet.from_dataset(
            self.training, predict_df, stop_randomization=True
        )

    def train_dataloader(self):
        return self.training.to_dataloader(
            batch_size=self.batch_size,
            shuffle=True,
        )

    def val_dataloader(self):
        return self.validation.to_dataloader(
            batch_size=self.batch_size,
            shuffle=False,
        )

    def test_dataloader(self):
        return self.test.to_dataloader(
            batch_size=self.batch_size,
            shuffle=False,
        )

    def predict_dataloader(self):
        return self.predict.to_dataloader(
            batch_size=self.batch_size,
            shuffle=False,
        )

#### Multivariate Time Series

In [ ]:
class MultivariateSeriesDataModule(pl.LightningDataModule):
    def __init__(
        self,
        data: pd.DataFrame,
        n_lags: int,
        horizon: int,
        test_size: float = 0.2,
        batch_size: int = 16,
        target_name: str = "",
    ):
        super().__init__()
        self.data = data
        self.feature_names = [col for col in data.columns if col != target_name]
        self.batch_size = batch_size
        self.test_size = test_size
        self.n_lags = n_lags
        self.horizon = horizon
        self.target_name = target_name
        self.target_scaler = MinMaxScaler()
        self.training: TimeSeriesDataSet
        self.validation: TimeSeriesDataSet
        self.test: TimeSeriesDataSet
        self.predict: TimeSeriesDataSet
        self.setup()

    def preprocess_data(self):
        self.data["target"] = self.data[self.target_name]
        self.data["time_index"] = np.arange(len(self.data))
        self.data["group_id"] = 0

    def split_data(self):
        time_indices = self.data["time_index"].values
        train_indices, test_indices = train_test_split(
            time_indices, test_size=self.test_size, shuffle=False
        )
        train_indices, val_indices = train_test_split(
            train_indices, test_size=0.1, shuffle=False
        )
        return train_indices, val_indices, test_indices

    def scale_target(self, df, indices):
        scaled_values = self.target_scaler.transform(df.loc[indices, ["target"]])
        df.loc[indices, "target"] = scaled_values

    def setup(self, stage=None):
        self.preprocess_data()
        train_indices, val_indices, test_indices = self.split_data()

        train_df = self.data.loc[self.data["time_index"].isin(train_indices)]
        val_df = self.data.loc[self.data["time_index"].isin(val_indices)]
        test_df = self.data.loc[self.data["time_index"].isin(test_indices)]
        predict_df = pd.concat([train_df, val_df, test_df])

        self.target_scaler.fit(train_df[["target"]])
        self.scale_target(train_df, train_df.index)
        self.scale_target(val_df, val_df.index)
        self.scale_target(test_df, test_df.index)

        train_df = train_df.drop(self.target_name, axis=1)
        val_df = val_df.drop(self.target_name, axis=1)
        test_df = test_df.drop(self.target_name, axis=1)

        self.training = TimeSeriesDataSet(
            train_df,
            time_idx="time_index",
            target="target",
            group_ids=["group_id"],
            max_encoder_length=self.n_lags,
            max_prediction_length=self.horizon,
            time_varying_unknown_reals=self.feature_names,
            scalers={name: MinMaxScaler() for name in self.feature_names},
        )
        self.validation = TimeSeriesDataSet.from_dataset(
            self.training, val_df, stop_randomization=True
        )
        self.test = TimeSeriesDataSet.from_dataset(
            self.training, test_df, stop_randomization=True
        )
        self.predict = TimeSeriesDataSet.from_dataset(
            self.training, predict_df, stop_randomization=True
        )

    def train_dataloader(self):
        return self.training.to_dataloader(batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self):
        return self.validation.to_dataloader(batch_size=self.batch_size, shuffle=False)

    def test_dataloader(self):
        return self.test.to_dataloader(batch_size=self.batch_size, shuffle=False)

    def predict_dataloader(self):
        return self.predict.to_dataloader(batch_size=1, shuffle=False)

### LSTM Model


In [ ]:
class MultivariateLSTM(pl.LightningModule):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim, lr=0.001):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.lr = lr
        self.lstm = nn.LSTM(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
        )
        self.fc1 = nn.Linear(in_features=hidden_dim, out_features=16)
        self.fc2 = nn.Linear(in_features=16, out_features=output_dim)

        self.mse = MeanSquaredError()
        self.mae = MeanAbsoluteError()
        self.evs = ExplainedVariance()
        self.r2 = R2Score()

    def forward(self, x):
        out, _ = self.lstm(x)
        out = out[:, -1, :]
        out = self.fc1(out)
        out = self.fc2(out)
        return out

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self(x["encoder_cont"])
        loss = F.mse_loss(y_pred, y[0])
        self.log("train_loss", loss)
        self.log("train_mse", self.mse(y_pred, y[0]))
        self.log("train_mae", self.mae(y_pred, y[0]))
        self.log("train_evs", self.evs(y_pred, y[0]))
        self.log("train_r2", self.r2(y_pred, y[0]))
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self(x["encoder_cont"])
        loss = F.mse_loss(y_pred, y[0])
        self.log("validation_loss", loss)
        self.log("validation_mse", self.mse(y_pred, y[0]))
        self.log("validation_mae", self.mae(y_pred, y[0]))
        self.log("validation_evs", self.evs(y_pred, y[0]))
        self.log("validation_r2", self.r2(y_pred, y[0]))
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self(x["encoder_cont"])
        loss = F.mse_loss(y_pred, y[0])
        self.log("test_loss", loss)
        self.log("test_mse", self.mse(y_pred, y[0]))
        self.log("test_mae", self.mae(y_pred, y[0]))
        self.log("test_evs", self.evs(y_pred, y[0]))
        self.log("test_r2", self.r2(y_pred, y[0]))
        return loss

    def predict_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self(x["encoder_cont"])
        y_pred = y_pred.detach().numpy()
        y_pred = self.trainer.datamodule.target_scaler.inverse_transform(y_pred)
        return y_pred

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.lr)

### CNN Model


In [ ]:
class MultivariateCNN(pl.LightningModule):
    def __init__(
        self,
        num_features: int,
        output_dim: int,
        hidden_dim,
        num_layers,
        lr: float = 0.001,
    ):
        super().__init__()
        self.lr = lr
        self.conv1d = nn.Conv1d(
            in_channels=num_features, out_channels=32, kernel_size=3
        )
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool1d(kernel_size=3)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(
            32 * ((num_features - 2) // 4), 64
        )  # 计算一维卷积和池化后的大小
        self.fc2 = nn.Linear(64, output_dim)

        self.mse = MeanSquaredError()
        self.mae = MeanAbsoluteError()
        self.evs = ExplainedVariance()
        self.r2 = R2Score()

    def forward(self, x):
        x = self.conv1d(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self(x["encoder_cont"].permute(0, 2, 1))
        loss = F.mse_loss(y_pred, y[0])
        self.log("train_loss", loss)
        self.log("train_mse", self.mse(y_pred, y[0]))
        self.log("train_mae", self.mae(y_pred, y[0]))
        self.log("train_evs", self.evs(y_pred, y[0]))
        self.log("train_r2", self.r2(y_pred, y[0]))
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self(x["encoder_cont"].permute(0, 2, 1))
        loss = F.mse_loss(y_pred, y[0])
        self.log("validation_loss", loss)
        self.log("validation_mse", self.mse(y_pred, y[0]))
        self.log("validation_mae", self.mae(y_pred, y[0]))
        self.log("validation_evs", self.evs(y_pred, y[0]))
        self.log("validation_r2", self.r2(y_pred, y[0]))
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self(x["encoder_cont"].permute(0, 2, 1))
        loss = F.mse_loss(y_pred, y[0])
        self.log("test_loss", loss)
        self.log("test_mse", self.mse(y_pred, y[0]))
        self.log("test_mae", self.mae(y_pred, y[0]))
        self.log("test_evs", self.evs(y_pred, y[0]))
        self.log("test_r2", self.r2(y_pred, y[0]))
        return loss

    def predict_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self(x["encoder_cont"].permute(0, 2, 1))
        y_pred = y_pred.detach().numpy()
        y_pred = self.trainer.datamodule.target_scaler.inverse_transform(y_pred)
        return y_pred

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.lr)

### Transformer Model

### Transformer-LSTM Model


#### Positional Encoding


In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(
            torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model)
        )
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer("pe", pe)

    def forward(self, x):
        x = x + self.pe[: x.size(0), :]
        return self.dropout(x)

#### Transformer Encoder


In [ ]:
class TransAm(pl.LightningModule):
    def __init__(self, feature_size=64, num_layers=6, dropout=0.1):
        super().__init__()
        self.save_hyperparameters()
        self.pos_encoder = PositionalEncoding(self.hparams.feature_size)
        self.encoder_layer = nn.TransformerEncoderLayer(
            d_model=self.hparams.feature_size, nhead=8, dropout=self.hparams.dropout
        )
        self.transformer_encoder = nn.TransformerEncoder(
            self.encoder_layer, num_layers=self.hparams.num_layers
        )
        self.decoder = nn.Linear(self.hparams.feature_size, 1)
        self.init_weights()

    def init_weights(self):
        initrange = 0.1
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src):
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src)
        return output

#### Attention


In [ ]:
class AttentionDecoder(pl.LightningModule):
    def __init__(self, code_hidden_size, hidden_size, time_step):
        super().__init__()
        self.save_hyperparameters()
        self.code_hidden_size = code_hidden_size
        self.hidden_size = hidden_size
        self.T = time_step

        self.attn1 = nn.Linear(
            in_features=2 * hidden_size, out_features=code_hidden_size
        )
        self.attn2 = nn.Linear(
            in_features=code_hidden_size, out_features=code_hidden_size
        )
        self.tanh = nn.Tanh()
        self.attn3 = nn.Linear(in_features=code_hidden_size, out_features=1)
        self.lstm = nn.LSTM(input_size=1, hidden_size=self.hidden_size, num_layers=1)
        self.tilde = nn.Linear(in_features=self.code_hidden_size + 1, out_features=1)
        self.fc1 = nn.Linear(
            in_features=code_hidden_size + hidden_size, out_features=hidden_size
        )
        self.fc2 = nn.Linear(in_features=hidden_size, out_features=1)

    def forward(self, h, y_seq):
        h_ = h.transpose(0, 1)
        batch_size = h.size(0)
        d = self.init_variable(1, batch_size, self.hidden_size)
        s = self.init_variable(1, batch_size, self.hidden_size)
        h_0 = self.init_variable(1, batch_size, self.hidden_size)
        h_ = torch.cat((h_0, h_), dim=0)

        for t in range(self.T):
            x = torch.cat((d, h_[t, :, :].unsqueeze(0)), 2)
            h1 = self.attn1(x)
            _, states = self.lstm(y_seq[:, t].unsqueeze(0).unsqueeze(2), (h1, s))
            d = states[0]
            s = states[1]
        y_res = self.fc2(self.fc1(torch.cat((d.squeeze(0), h_[-1, :, :]), dim=1)))
        return y_res

    def init_variable(self, *args):
        zero_tensor = torch.zeros(args)
        return Variable(zero_tensor)

    def embedding_hidden(self, x):
        return x.permute(1, 0, 2)

#### Transformer-LSTM Model


In [ ]:
class TransformerLSTM(pl.LightningModule):
    def __init__(self, time_step=10, lr=0.001):
        super().__init__()
        self.time_step = time_step
        self.lr = lr
        self.encoder = TransAm()
        self.decoder = AttentionDecoder(
            code_hidden_size=64, hidden_size=64, time_step=self.time_step
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded, x)
        return decoded

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self(x["encoder_cont"])
        y_pred = y_pred.squeeze(1)
        loss = F.mse_loss(y_pred, y[0])
        self.log("train_loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self(x["encoder_cont"])

        y_pred = y_pred.squeeze(1)

        loss = F.mse_loss(y_pred, y[0])
        self.log("val_loss", loss)
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self(x["encoder_cont"])
        y_pred = y_pred.squeeze(1)
        loss = F.mse_loss(y_pred, y[0])
        self.log("test_loss", loss)

    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        x, y = batch
        y_pred = self(x["encoder_cont"])
        y_pred = y_pred.squeeze(1)
        return y_pred

### SVR 模型，回归


In [ ]:
stock_index = 4
data = stock_market[stock_index]
X, y = data[features_two], data["close"]

# 将数据集分成前70%作为训练集，后30%作为测试集
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, shuffle=False
)
# 特征标准化
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
# 构建SVR模型
svr = SVR(kernel="linear")
svr.fit(X_train, y_train)
# 预测
y_pred = svr.predict(X_test)
# 评估模型
# 计算均方误差（MSE）
mse = mean_squared_error(y_test, y_pred)
# 平均绝对误差（MAE）
mae = mean_absolute_error(y_test, y_pred)
# 均方对数误差 (MSLE)
msle = mean_squared_log_error(y_test, y_pred)
# 中值绝对误差 (MedAE)
medae = median_absolute_error(y_test, y_pred)
# 解释方差分数
evs = explained_variance_score(y_test, y_pred)
# R2 分数（决定系数）
R2 = r2_score(y_test, y_pred)

# 将指标存储在字典中
metrics = {
    "Mean Squared Error": mse,
    "Mean Absolute Error": mae,
    "Median Absolute Error": medae,
    "Explained Variance Score": evs,
    "R2 Score": R2,
}

# 创建一个DataFrame
df_metrics = pd.DataFrame(list(metrics.items()), columns=["Metric", "Value"])
df_metrics["Value"] = df_metrics["Value"].round(2)
# 显示DataFrame
print(df_metrics)

# 可视化预测结果和实际数据
plt.figure(figsize=(20, 6))
plt.plot(y_test.index, y_test, label="Actual Values")
plt.plot(y_test.index, y_pred, label="Predicted Values")
plt.xlabel("Date")
plt.ylabel("Index Values")
plt.title("Prediction vs Actual" + " by SVM Model - " + stock_name[stock_index])
plt.legend()
plt.show()

### 道琼斯指数


#### 检查数据集


In [ ]:
stock_index = 2
data = stock_market[stock_index]

datamodule = MultivariateSeriesDataModule(
    data=data[features_one + ["Close"]],
    n_lags=10,
    horizon=1,
    test_size=0.2,
    target_name="Close",
    batch_size=16,
)

train_dataloader = datamodule.train_dataloader()
x, y = next(iter(train_dataloader))
# print("x =", x)
# print("\ny =", y)
print("\nsizes of x =")
for key, value in x.items():
    print(f"\t{key} = {value.size()}")

#### 检查模型


In [ ]:
model = MultivariateLSTM(
    input_dim=len(features_one), hidden_dim=64, num_layers=2, output_dim=1, lr=0.001
)
summary(model, input_size=(16, 10, len(features_one)))

In [ ]:
logger = TensorBoardLogger("lightning_logs", name="downjones")
# early_stop_callback = EarlyStopping(
#     monitor="validation_loss", min_delta=1e-4, patience=10, verbose=False, mode="min"
# )
# model_checkpoint = ModelCheckpoint(
#     dirpath="checkpoints/downjones",
#     filename="{epoch}-{val_loss:.2f}-{other_metric:.2f}",
# )
rich_progress_bar = RichProgressBar()
trainer = pl.Trainer(
    logger=logger,
    # callbacks=[model_checkpoint, early_stop_callback, rich_progress_bar],
    callbacks=[rich_progress_bar],
    max_epochs=500,
    accelerator="cpu",
)
trainer.fit(model=model, datamodule=datamodule)

In [ ]:
trainer.test(model=model, datamodule=datamodule)

In [ ]:
trainer.validate(model=model, datamodule=datamodule)

In [ ]:
prediction = trainer.predict(model=model, datamodule=datamodule)

In [ ]:
prediction = np.concatenate(prediction)
prediction = prediction.flatten()

In [ ]:
actual_final = data["Close"].values[-len(prediction) :]

In [ ]:
temp = pd.DataFrame(zip(prediction, actual_final), columns=["y_pred", "test"])
# 可视化预测结果和实际数据
plt.figure(figsize=(20, 6))
plt.plot(temp.index, temp["test"], label="Actual Values")
plt.plot(temp.index, temp["y_pred"], label="Predicted Values")
plt.xlabel("Date")
plt.ylabel("Index Values")
plt.title("Prediction vs Actual" + " by LSTM Model - " + stock_name[stock_index])
plt.legend()
plt.grid()
plt.show()

### 上证指数


#### 检查数据集


In [ ]:
stock_index = 4
data = stock_market[stock_index]

datamodule = MultivariateSeriesDataModule(
    data=data[features_two + ["close"]],
    n_lags=10,
    horizon=1,
    test_size=0.2,
    target_name="close",
    batch_size=16,
)

train_dataloader = datamodule.train_dataloader()
x, y = next(iter(train_dataloader))
# print("x =", x)
# print("\ny =", y)
print("\nsizes of x =")
for key, value in x.items():
    print(f"\t{key} = {value.size()}")

#### 检查模型


In [ ]:
model = MultivariateCNN(
    num_features=len(features_two), hidden_dim=64, num_layers=1, output_dim=1, lr=0.001
)
# model
summary(model, input_size=(16, len(features_two), 10))

In [ ]:
logger = TensorBoardLogger("lightning_logs", name="Shanghai-Composite-Index")
# early_stop_callback = EarlyStopping(
#     monitor="validation_loss", min_delta=1e-4, patience=10, verbose=False, mode="min"
# )
# model_checkpoint = ModelCheckpoint(
#     dirpath="checkpoints/downjones",
#     filename="{epoch}-{val_loss:.2f}-{other_metric:.2f}",
# )
rich_progress_bar = RichProgressBar()
trainer = pl.Trainer(
    logger=logger,
    # callbacks=[model_checkpoint, early_stop_callback, rich_progress_bar],
    callbacks=[rich_progress_bar],
    max_epochs=500,
    accelerator="cpu",
)
trainer.fit(model=model, datamodule=datamodule)

In [ ]:
trainer.test(model, datamodule=datamodule)

In [ ]:
trainer.validate(model=model, datamodule=datamodule)

In [ ]:
prediction = trainer.predict(model=model, datamodule=datamodule)

In [ ]:
scaler = datamodule.target_scaler

In [ ]:
prediction = np.concatenate(prediction)
prediction = prediction.flatten()

In [ ]:
actual_final = data["close"].values[-len(prediction) :]

In [ ]:
temp = pd.DataFrame(zip(prediction, actual_final), columns=["y_pred", "test"])
# 可视化预测结果和实际数据
plt.figure(figsize=(20, 6))
plt.plot(temp.index, temp["test"], label="Actual Values")
plt.plot(temp.index, temp["y_pred"], label="Predicted Values")
plt.xlabel("Date")
plt.ylabel("Index Values")
plt.title("Prediction vs Actual" + " by CNN Model - " + stock_name[stock_index])
plt.legend()
plt.grid()
plt.show()

### 标准普尔 500 指数


#### 检查数据集


In [ ]:
stock_index = 3
data = stock_market[stock_index]

datamodule = UnivariateSeriesDataModule(
    data=data,
    n_lags=10,
    horizon=1,
    test_size=0.2,
    batch_size=16,
    feature_name="Close",
)

train_dataloader = datamodule.train_dataloader()
x, y = next(iter(train_dataloader))
# print("x =", x)
# print("\ny =", y)
print("\nsizes of x =")
for key, value in x.items():
    print(f"\t{key} = {value.size()}")

#### 检查模型

In [ ]:
model = TransformerLSTM(
    time_step=10,
    lr=0.001,
)
model
# summary(model, input_size=(16, len(features_two), 10))